In [3]:
%pip install xlrd==2.0.1

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: C:\Users\Elena\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [4]:
%pip install pywin32

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: C:\Users\Elena\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [5]:
%pip install unidecode

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: C:\Users\Elena\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [15]:
%pip install rapidfuzz

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 1.5/1.5 MB 18.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: C:\Users\Elena\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


## 0. Carga de Librerías

In [1]:
# Cargamos las librerias necesarias
import re
import pandas as pd
from pathlib import Path
from unidecode import unidecode


## 1. Definición de FUNCIÓN que NORMALIZA nombres para merge tablas. 


In [ ]:

# 1) Normalizador de nombres
# ----------------------------

def normaliza_nombre(nombre: str) -> str:
    if pd.isna(nombre):
        return ""
    # Pasar a minúsculas y quitar espacios al inicio/final
    s = str(nombre).lower().strip()

    # Unificar abreviaturas frecuentes
    s = re.sub(r"\bs\.\b", " san ", s)  # "S." -> "san"
    s = re.sub(r"\bsta\.\b", " santa ", s)  # "Sta." -> "santa"
    s = re.sub(r"\bsto\.\b", " santo ", s)  # "Sto." -> "santo"

    # Quitar tildes/diacríticos
    s = unidecode(s)

    # Reemplazos de conectores frecuentes
    s = re.sub(r"[-_/.,;:()]", " ", s)  # puntuación -> espacio

    # Normalizar espacios
    s = re.sub(r"\s+", " ", s).strip()

    # Quitar artículos/preposiciones sueltas al inicio/medio (palabras aisladas)
    ARTICULOS_ES = {"el", "la", "los", "las", "de", "del", "y", "da", "do", "das", "dos"}
    ARTICULOS_CAT = {"l", "l'", "els", "les", "es", "sa", "ses", "s'", "son"}
    # Nota: NO quitamos 'san', 'santa', 'santo', 'sant' porque son parte del nombre.
    # Sí expandimos 's.' a 'san' abajo.
    tokens = [t for t in s.split() if t not in ARTICULOS_ES | ARTICULOS_CAT]
    s = " ".join(tokens)

    # Normalizaciones finas
    s = re.sub(r"\bde el\b", " del", s)  # por si quedó "de el"
    s = re.sub(r"\s+", " ", s).strip()

    return s
#

## 2. Definición de FUNCIÓN que MERGE por municipio la POTENCIA INSTALADA para dar las coordenadas. 

In [3]:
# ----------------------------
# 2) MERGE las instalaciones con coordenadas
# ----------------------------
def instalaciones_coords(df_mun, df_ins,   
    col_muni_mun="NOMBRE_ACTUAL",
    col_lat="LATITUD_ETRS89",
    col_lon="LONGITUD_ETRS89",
    col_muni_inst="Municipio",

):
       
    # Claves normalizadas
    df_mun["__mun_key__"] = df_mun[col_muni_mun].apply(normaliza_nombre)
    df_ins["__mun_key__"] = df_ins[col_muni_inst].apply(normaliza_nombre)

    # Quitar duplicados de municipios por clave normalizada conservando el primero
    dups = df_mun["__mun_key__"].duplicated(keep=False)
    if dups.any():
        #print("Aviso: hay municipios duplicados tras normalizar. Se conservará el primero por clave:")
        # print(df_mun.loc[dups, [col_muni_mun, "__mun_key__"]].sort_values("__mun_key__").to_string(index=False))
        df_mun = df_mun.drop_duplicates(subset="__mun_key__", keep="first")

    # Merge LEFT: todas las instalaciones + lat/lon del municipio
    df_merged = df_ins.merge(
        df_mun[["__mun_key__", col_lat, col_lon]],
        on="__mun_key__",
        how="left",
    )

    # Reporte de match
    total = len(df_ins)
    matched = df_merged[col_lat].notna().sum()
    print(f" Emparejadas: {matched:,}/{total:,} ({matched/total:,.1%})")

    # No emparejadas
    sin_match = df_merged[df_merged[col_lat].isna()][[col_muni_inst, "__mun_key__"]].drop_duplicates()
    if not sin_match.empty:
        print("\n🔎 Sin match (ejemplos):")
        # print(sin_match.head(20).to_string(index=False))
        # sugiere_matches(df_ins, df_mun, col_muni_inst="Municipio", top_n=3, umbral=85)
        
    return df_merged



## 3. LLAMADA a funciones y definción de directorios

In [5]:
# Directorio donde está el notebook
BASE_DIR = Path.cwd()
# Carpeta padre (un nivel arriba)
PARENT_DIR = BASE_DIR.parent

# Definición de directorios de carga y guardado de datos
directorio_0 = PARENT_DIR / "08. Municipios BD"
directorio_2 = PARENT_DIR / "05. Registro Instalaciones"
directorio_3= PARENT_DIR / "13. Mallado Potencia Instalada"

# Cargar datos
df_mun = pd.read_csv(f"{directorio_0}/MUNICIPIOS.csv", sep=';', encoding='latin-1')
df_ins = pd.read_csv(f"{directorio_2}/PRETOR_REDUCIDO.csv", sep=',', encoding='latin-1')

#Llama a función para unir las tablas
df_instalaciones_coords = instalaciones_coords(df_mun=df_mun, df_ins=df_ins)

# Guardar resultados
df_instalaciones_coords.to_excel(f"{directorio_3}/PRETOR_REDUCIDO_Coord.xlsx", index=False, engine="openpyxl")
df_instalaciones_coords.to_csv(f"{directorio_3}/PRETOR_REDUCIDO_Coord.csv", index=False)


 Emparejadas: 37,419/48,942 (76.5%)

🔎 Sin match (ejemplos):
